In [10]:
# contract always success
!echo -e "int main() {\n\
    return 0;\n\
}" > contracts/always_success.c

In [11]:
!riscv64-unknown-elf-gcc -O3 -mcmodel=medlow -Wall -Werror -Wno-nonnull-compare -Wno-unused-function -Wl,-static -fdata-sections -ffunction-sections -Wl,--gc-sections -Wl,-s -o build/always_success contracts/always_success.c

In [12]:
# connect rpc wrapper
from tinyrpc.protocols.jsonrpc import JSONRPCProtocol
from tinyrpc.transports.http import HttpPostClientTransport
from tinyrpc import RPCClient

rpc_client = RPCClient(
    JSONRPCProtocol(),
    HttpPostClientTransport('http://127.0.0.1:8999/'))

remote_server = rpc_client.get_proxy()

remote_server.blockNumber()

4304

In [13]:
# index 0 user for delpoy contract
remote_server.getHDUserInfo(0)

{'privkey': '0xbb918557dad030f8c16d2cebb3da66d380e11ede53214d6e8097d4967bed9d3c',
 'pubkey': '0x025575c6dc037c40b2f7e5cccdb2f071135fe60dfbe1c85bf5a6052734bab7eaaa',
 'blake160': '0x32d0281300319169a9c6f01f1bd6ee566089bdfe',
 'address': 'ckt1qyqr95pgzvqrrytf48r0q8cm6mh9vcyfhhlq4hj46t'}

In [14]:
!pwd

/home/rink/work/github/ckb-notebook


In [16]:
# send some capacity to account index 0 
remote_server.deployContract("0xbb918557dad030f8c16d2cebb3da66d380e11ede53214d6e8097d4967bed9d3c" ,"/home/rink/work/github/ckb-notebook/build/always_success")

JSONRPCError: Server application error